In [3]:
# connector de python à mysql
import mysql.connector
import json
#**************************************************************************************CREATION DE LA CONNEXION SQLALCHEMY**********************************************************

## IMPORT DU CONFIG.JSON
# assignation de la config.json à fichierConfig
fichierConfig = "config.json"
# ouverture et chargement des donnée contenu dans fichierConfig
with open(fichierConfig) as fichier:
    config = json.load(fichier)["mysql"]

# assignation de de la connexion par create_engine avec les éléement de connexion + les info du fichierConfig à engine
# en dehors de la class car cet élémeent ne change pas
connection = mysql.connector.connect(host=config["host"],database = config["bdd"],user=config["user"],password=config["password"])
cursor = connection.cursor(buffered=True)

In [4]:
mail = "Salut, J’ai besoin de la salle 736 le 17 janvier entre 8 h et 30 h"

In [13]:
class CheckReservation():

    def __init__(self, mail):
        self.message = mail
        self.salle = self._ParseMailSalle()
        self.date = self._ParseMailDate()
        self.horraire = self._ParseMailHorraire()
        self.check_horraire = True if self.horraire[0] in range(0,25) and self.horraire[1] in range(0,25) else False
        self.check_salle_exist = self._CheckSalleExist()
        self.check_horraire_dispo = self._CheckHorraireDispo()
        
        #self.list_message = ["Non valide","Changer horraire"]
        #self.message = #fonction qui deduit le message erreur
################################################################################### MOTIF SALLE ##############################################################


    def _ParseMailSalle(self):
        import re
        motif_salle = (r"(?:salle+\s+\d+)|(?:Salle+\s+\d+)")
        res_mail_salle = re.findall(motif_salle,self.message) # application de la méthode findall() avec regex, on applique le motif sur le "mail_contenu", stocker dans la variable "res_mail"
        num_salle_reserv = str(res_mail_salle[0])
        num_salle_reserv = str(num_salle_reserv)
        
        
        return num_salle_reserv


################################################################################### MOTIF DATE ##############################################################


    def _ParseMailDate(self):
        import re
        import dateparser
        import datetime

        motif_date = (r"(?:\d+\s+janvier)|(?:\d+\s+fevrier)|(?:\d+\s+mars)|(?:\d+\s+avril)|(?:\d+\s+mai)|(?:\d+\s+juin)|(?:\d+\s+juillet)|(?:\d+\s+août)|(?:\d+\s+septembre)|(?:\d+\s+octobre)|(?:\d+\s+novembre)|(?:\d+\s+decembre)|(?:\d+\s+Janvier)|(?:\d+\s+Fevrier)|(?:\d+\s+Mars)|(?:\d+\s+Avril)|(?:\d+\s+Mai)|(?:\d+\s+Juin)|(?:\d+\s+Juillet)|(?:\d+\s+Août)|(?:\d+\s+Septembre)|(?:\d+\s+Octobre)|(?:\d+\s+Novembre)|(?:\d+\s+Decembre)|(?:\d+\s+January)|(?:\d+\s+February)|(?:\d+\s+March)|(?:\d+\s+April)|(?:\d+\s+May)|(?:\d+\s+June)|(?:\d+\s+July)|(?:\d+\s+Augut)|(?:\d+\s+September)|(?:\d+\s+October)|(?:\d+\s+November)|(?:\d+\s+December)|(?:\d+\s+january)|(?:\d+\s+february)|(?:\d+\s+march)|(?:\d+\s+april)|(?:\d+\s+Mai)|(?:\d+\s+june)|(?:\d+\s+july)|(?:\d+\s+august)|(?:\d+\s+september)|(?:\d+\s+october)|(?:\d+\s+november)|(?:\d+\s+december)")  #### (?:\d+.\d+) 12/03 pour la date format numérique
        res_mail_date = re.findall(motif_date, self.message)
        date_reserv = str(res_mail_date[0])
        date_reserv = str(date_reserv)
        date_reserv = dateparser.parse(date_reserv)  
        now = datetime.datetime.now()   
            
        if date_reserv <= now:
            now_year = now.year + 1     
        else:                           
            now_year = now.year                   
            
        date_reserv = str(date_reserv)  
        date_reserv = date_reserv[5:10] 
        date_reserv = str(now_year) + ' ' + date_reserv  
        date_reserv = dateparser.parse(date_reserv)
        date_reserv = date_reserv.date()



        return date_reserv

################################################################################### MOTIF HORRAIRE ##############################################################
    

    def _ParseMailHorraire(self):
        import re
        motif_horraire = (r"(?:\d+.Heures)|(?:\d+.Heure)|(?:\d+.heures)|(?:\d+.heure)|(?:\d+.h)|(?:\d+.H)")
        res_mail_horraire = re.findall(motif_horraire,self.message) 
        
        debut_reserv = res_mail_horraire[0][0:2]
        debut_reserv = int(debut_reserv)
        
        fin_reserv = res_mail_horraire[1][0:2]
        fin_reserv = int(fin_reserv)   
        
        time_reserv = fin_reserv - debut_reserv


        return debut_reserv, fin_reserv, time_reserv



################################################################################### CHECK SALLE EXIST  ##############################################################


    def _CheckSalleExist(self):
        check_salle_exist = cursor.callproc('PSGetSALLEbyNAME',(self.salle,))[0]
        return True if check_salle_exist != None else False
            

################################################################################### CHECK HORRAIRE DISPO  ##############################################################


    def _CheckHorraireDispo(self):

        cursor.execute('SELECT res_id, res_date, res_heure_arrive, res_heure_depart, S.sa_name FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND(( %s BETWEEN R.res_heure_arrive AND R.res_heure_depart ) OR (%s BETWEEN R.res_heure_arrive AND R.res_heure_depart));', (self.salle,self.date,self.horraire[0],self.horraire[1]))

        resa_db_1 = cursor.fetchone()
        
        result = True if resa_db_1 == None else False


        if not result:
            if resa_db_1 != None and self.horraire[1] == resa_db_1[3]:
                cursor.execute('SELECT * FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND ((%s > R.res_heure_arrive AND %s < R.res_heure_depart) OR (%s >= R.res_heure_arrive AND %s <= R.res_heure_depart));', (self.salle,self.date,self.horraire[0],self.horraire[1]))
                result_fin_to_debut_reserv = cursor.fetchone()
    
                if result_fin_to_debut_reserv == None and resa_db_1 != None:
                    result = True
    



            elif resa_db_1 != None and self.horraire[0] == resa_db_1[2]:
                cursor.execute('SELECT * FROM RESERVATION R INNER JOIN SALLE S ON S.sa_id=R.sa_id WHERE S.sa_name=%s AND R.res_date=%s AND ((%s >= R.res_heure_arrive AND %s <= R.res_heure_depart) OR (%s > R.res_heure_arrive AND %s < R.res_heure_depart));', (self.salle,self.date,self.horraire[0],self.horraire[0],self.horraire[1],self.horraire[1]))
                result_debut_to_fin_reserv = cursor.fetchone()
    
                if result_debut_to_fin_reserv == None and resa_db_1 != None:
                    result = True


            else:
                result = False#,"TESTING NOT OK 1"


        return result

############################################################################################################################################################################


    def _InsertResa(self):

        if result == True and salle_valide == True:             
            
        parameter_ma_contenu = mail_contenu ## on initialise le mail_contenu dans la variable "parameter_ma_contenu"
        parameter_session_id = session['id'] ## on initialise le session['id'] qui est l'id de la session en cours dans la variable "parameter_session_id"

        cursor.callproc("PI_MAIL_SIMPLE", [parameter_ma_contenu,parameter_session_id,],)    ## ON INSERE DANS LA TABLE MAIL LE MAIL CONTENU ET L'ID DE LA SESSION
        connection.commit() ## ON VALIDE L'INSERTION
                
                ### recup ma_id du mail inserer dans le dernier ma_contenu de l'user
        cursor.execute('SELECT ma_id FROM MAIL WHERE in_id = %s ORDER BY ma_date DESC LIMIT 1 ', (parameter_session_id,))   ## ON RECUPERE LE DERNIER MAIL ENVOYER DE L'USER ORDONNER PAR LA DATE ET L'HEURE ET SECONDE LA PLUS RECENTE 
        ma_id_recup = cursor.fetchone() ## ON STOCKE LE RESULTAT DE LA REQUETE DANS LA VARIABLE "ma_id_recup"
        ma_id_recup = ma_id_recup[0]    ## ma_id_recup est initialise avec son premier élément de sa liste
                
                
                ### recup le sa_id where num_salle_reserv
        cursor.execute('SELECT sa_id FROM SALLE WHERE sa_name = %s ', (num_salle_reserv,)) ## ON REQUETE LA DATABASE POUR RECUPER L'ID DE LA SALLE DEMANDER PAR LE NOM DE LA SALLE
        sa_id_recup = cursor.fetchone() # on stocke le resultat de la requete dans "sa_id_recup"
        sa_id_recup = str(sa_id_recup[0])    # sa_id_recup est initialise avec son premier élément de sa liste
                    
                    


        if sa_id_recup != None:

        args1 = date_reserv
        args2 = debut_reserv
        args3 = fin_reserv
        args4 = time_reserv
        args5 = sa_id_recup
        args6 = ma_id_recup
                
                
        cursor.callproc("PI_RES_SIMPLE", (args1,args2,args3,args4,args5,args6,))    ### EXECUTION DE LA PROCEDURE STOCKE POUR INSERER LA RESERVATION
        connection.commit()     ## ON VALIDE L'INSERTION
                
        msg = 'You have successfully send your reservation !'       ## ON AVERTIE L'USER qu'il a bien envoyer sa reservation 
        print(msg)


In [34]:
testclass = CheckReservation(mail)

print(10 * "*")
print("date de la reservation", '\n')
print(testclass.date , '\n')

print(10 * "*")
print("Heure debut de la reservation", '\n')

print(testclass.horraire[0] , '\n')

print(10 * "*")
print("Heure fin de la reservation", '\n')
print(testclass.horraire[1] , '\n')

print(10 * "*")
print("Durée de la reservation", '\n')

print(testclass.horraire[2] , '\n')

print(10 * "*")
print("Check si les horaire sont valide", '\n')

print(testclass.check_horraire , '\n')

print(10 * "*")
print("Check si la salle existe", '\n')
print(testclass.check_salle_exist , '\n')

print(10 * "*")
print("Check si les horaire demandé sont dispo", '\n')
print(testclass.check_horraire_dispo , '\n')

print(10 * "*")

**********
date de la reservation 

2021-01-17 

**********
Heure debut de la reservation 

8 

**********
Heure fin de la reservation 

30 

**********
Durée de la reservation 

22 

**********
Check si les horaire sont valide 

False 

**********
Check si la salle existe 

True 

**********
Check si les horaire demandé sont dispo 

False 

**********
